In [41]:
%load_ext autoreload
%autoreload 2

from utils import *
from dataset_prepare import *

import pickle

from matplotlib import pyplot as plt

from aif360.metrics import BinaryLabelDatasetMetric
from aif360.datasets.standard_dataset import StandardDataset
from aif360.algorithms.preprocessing import Reweighing, DisparateImpactRemover
from aif360.metrics import ClassificationMetric
from sklearn.metrics import classification_report
from aif360.algorithms.preprocessing.optim_preproc_helpers.opt_tools import OptTools

from IPython.display import Markdown, display

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import balanced_accuracy_score, accuracy_score

from tqdm import tqdm
import imblearn

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [42]:
df = load_dataset()
df

,Num_Acc,trajet,catr,circ,nbv,prof,plan,surf,infra,situ,...,atm,col,catv,obs,obsm,choc,mortal,pieton,sexe_conducteur,age
0,202200000001,5,4,2,2,1,1,1,0,1,...,1,3,2,0,2,1,0,0,1,14
1,202200000001,5,4,2,2,1,1,1,0,1,...,1,3,2,0,2,1,0,0,1,74
2,202200000002,0,4,2,2,1,1,1,0,1,...,1,3,3,0,2,8,0,0,1,34
3,202200000002,4,4,2,2,1,1,1,0,1,...,1,3,3,0,2,8,0,0,1,52
4,202200000003,0,3,-1,2,1,1,1,5,1,...,1,2,3,0,2,1,0,0,1,20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
126656,202200055300,5,3,2,2,1,2,7,0,3,...,9,6,3,1,0,1,1,0,1,27
126657,202200055301,5,3,2,2,1,1,1,0,1,...,1,3,3,0,0,8,0,0,0,20
126659,202200055301,5,3,2,2,1,1,1,0,1,...,1,3,3,0,0,8,0,0,0,69
126660,202200055302,1,3,3,4,1,1,1,0,1,...,1,2,5,0,2,1,0,0,1,30


In [43]:
label = 'mortal'

# Récupération des ensembles de train/test
X_train, X_test, y_train, y_test = test_train_sets(df)
df = df.drop(columns='Num_Acc')

In [44]:
dataset_orig_train, dataset_orig_test = prepare_standard_dataset(X_train, y_train, X_test, y_test, label)

In [45]:
# print out some labels, names, etc.
display(Markdown("#### Training Dataset shape"))
print(dataset_orig_train.features.shape)
display(Markdown("#### Favorable and unfavorable labels"))
print(dataset_orig_train.favorable_label, dataset_orig_train.unfavorable_label)
display(Markdown("#### Protected attribute names"))
print(dataset_orig_train.protected_attribute_names)
display(Markdown("#### Privileged and unprivileged protected attribute values"))
print(dataset_orig_train.privileged_protected_attributes, 
      dataset_orig_train.unprivileged_protected_attributes)
display(Markdown("#### Dataset feature names"))
print(dataset_orig_train.feature_names)

#### Training Dataset shape

(117678, 24)


#### Favorable and unfavorable labels

1.0 0.0


#### Protected attribute names

['sexe_conducteur']


#### Privileged and unprivileged protected attribute values

[array([1.])] [array([0.])]


#### Dataset feature names

['trajet', 'catr', 'circ', 'nbv', 'prof', 'plan', 'surf', 'infra', 'situ', 'vma', 'mois', 'lum', 'dep', 'agg', 'int', 'atm', 'col', 'catv', 'obs', 'obsm', 'choc', 'pieton', 'sexe_conducteur', 'age']


In [46]:
privileged_groups=[{'sexe_conducteur': 1}]
unprivileged_groups = [{'sexe_conducteur' : 0}]

## Repondération
Utilisation de l'outil de repondération de `aif360` et affichage du résultat obtenu.

In [47]:
# Metric for the original dataset
metric_orig_train = BinaryLabelDatasetMetric(dataset_orig_train, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)
display(Markdown("#### Original training dataset"))
print("Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_orig_train.mean_difference())

#### Original training dataset

Difference in mean outcomes between unprivileged and privileged groups = 0.080457


In [48]:
# 0 -> Femme et 1 -> Homme
RW = Reweighing(privileged_groups=privileged_groups, unprivileged_groups=unprivileged_groups)
RW.fit(dataset_orig_train)
dataset_transf_train = RW.transform(dataset_orig_train)

In [49]:
metric_transf_train = BinaryLabelDatasetMetric(dataset_transf_train, 
                                         unprivileged_groups=unprivileged_groups,
                                         privileged_groups=privileged_groups)
display(Markdown("#### Transformed training dataset"))
print("Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_transf_train.mean_difference())

#### Transformed training dataset

Difference in mean outcomes between unprivileged and privileged groups = 0.000000


In [50]:
# valeurs catégorielles
categorical_features = ['trajet', 'catr', 'circ', 'nbv', 'prof',
                        'plan', 'surf', 'vma', 'lum', 'agg', 
                        'int', 'atm', 'col', 'catv', 'obs', 'obsm', 'choc', 'pieton',
                        'sexe_conducteur', 'infra', 'situ']
# valeurs numériques
numerical_features = ['dep','age', 'mois']

print("numerical : ", numerical_features)
print("categorical : ", categorical_features)

numerical :  ['dep', 'age', 'mois']
categorical :  ['trajet', 'catr', 'circ', 'nbv', 'prof', 'plan', 'surf', 'vma', 'lum', 'agg', 'int', 'atm', 'col', 'catv', 'obs', 'obsm', 'choc', 'pieton', 'sexe_conducteur', 'infra', 'situ']


In [51]:
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())])

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

transformations = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)])

In [52]:
X_train = dataset_transf_train.convert_to_dataframe()[0].drop(columns=['mortal'])
y_train = dataset_transf_train.convert_to_dataframe()[0]['mortal']
X_test = dataset_orig_test.convert_to_dataframe()[0].drop(columns=['mortal'])
y_test = dataset_orig_test.convert_to_dataframe()[0]['mortal']

In [53]:
from utils import custom_RFC
import imblearn

dt = custom_RFC(random_state=42)
clf = imblearn.pipeline.Pipeline(
    [
        ('preprocessor', transformations),
        ('classifier', dt)
    ])
clf = clf.fit(X_train, y_train, classifier__sample_weight=dataset_orig_train.instance_weights)

preds = clf.predict(X_test)

In [56]:
print("Accuracy : ", clf.score(X_train, y_train), clf.score(X_test, y_test) )

Accuracy :  0.9891398562178148 0.7555738243998684


In [57]:
pickle.dump(clf, open('models/rfc_reweight_model.sav', 'wb'))

In [58]:
print_conf_matrix(y_test, preds)

In [59]:
print_metrics(clf, X_test, y_test, df, categorical_features)

sexe_conducteur 0
disparate_impact 0.9492182363968904 0.6355715740529334
P_rule_disparate_impact 0.9492182363968904 0.6355715740529334
demography_parity -0.01313048010351689 -0.022439373005842936


Les femmes conductrices impliquées dans un accident mortel : 0.6038338658146964 
Les femmes conductrices impliquées dans un accident non mortel : 0.23083929733246583 
Les hommes conducteurs impliquées dans un accident mortel : 0.5942028985507246 
Les hommes conducteurs impliquées dans un accident non mortel : 0.23654431342715862 



## Disparate impact remover
Utilisation de l'outil de `Disparate impact remover` de `aif360` et affichage du résultat obtenu.

In [20]:
label = 'mortal'

df = load_dataset()
# Récupération des ensembles de train/test
X_train, X_test, y_train, y_test = test_train_sets(df)

In [22]:
# Récupération StandardDataset
dataset_orig_train, dataset_orig_test = prepare_standard_dataset(X_train, y_train, X_test, y_test, label, transform=True)

### Évaluation du disparate impact improver pour différentes valeurs du paramètre `level`

In [23]:
protected = 'sexe_conducteur'

index = dataset_orig_train.feature_names.index(protected)

DIs = []
for level in tqdm(np.linspace(0., 1., 11)):
    di = DisparateImpactRemover(repair_level=level)
    train_repd = di.fit_transform(dataset_orig_train)
    test_repd = di.fit_transform(dataset_orig_test)

    X_tr = np.delete(train_repd.features, index, axis=1)
    X_te = np.delete(test_repd.features, index, axis=1)
    
    y_tr = train_repd.labels.ravel()
    
    dt = custom_RFC(random_state=42)
    dt.fit(X_tr, y_tr)
    
    test_repd_pred = test_repd.copy()
    test_repd_pred.labels = dt.predict(X_te)

    p = [{protected: 1}]
    u = [{protected: 0}]
    cm = BinaryLabelDatasetMetric(test_repd_pred, privileged_groups=p, unprivileged_groups=u)
    DIs.append(cm.disparate_impact())

  0%|          | 0/11 [00:00<?, ?it/s]

100%|██████████| 11/11 [08:22<00:00, 45.67s/it]


In [24]:
plt.plot(np.linspace(0, 1, 11), DIs, marker='o')
plt.plot([0, 1], [1, 1], 'g')
plt.plot([0, 1], [0.8, 0.8], 'r')
plt.ylim([0.4, 1.2])
plt.ylabel('Disparate Impact (DI)')
plt.xlabel('repair level')
plt.show()
plt.savefig("disparate_impact_remover.png")

/tmp/ipykernel_61413/3631253355.py:7: UserWarning:

FigureCanvasAgg is non-interactive, and thus cannot be shown



### Évaluation des résultats avec le meilleur paramètre trouvé

In [25]:
protected = 'sexe_conducteur'

index = dataset_orig_train.feature_names.index(protected)

di = DisparateImpactRemover(repair_level=0.9)
train_repd = di.fit_transform(dataset_orig_train)
test_repd = di.fit_transform(dataset_orig_test)

X_tr = np.delete(train_repd.features, index, axis=1)
X_te = np.delete(test_repd.features, index, axis=1)

y_tr = train_repd.labels.ravel()
y_te = test_repd.labels.ravel()

dt = custom_RFC(random_state=42)
dt.fit(X_tr, y_tr)

test_repd_pred = test_repd.copy()
preds = dt.predict(X_te)


In [31]:
dt.score(X_tr, y_tr), dt.score(X_te, y_te)

(0.9887149679634256, 0.7598816178888523)

In [32]:
X_train = train_repd.convert_to_dataframe()[0]
X_test = test_repd.convert_to_dataframe()[0]
y_train = train_repd.convert_to_dataframe()[0]['mortal']
y_test = test_repd.convert_to_dataframe()[0]['mortal']

In [34]:
print_conf_matrix(y_test, preds)

In [40]:
# Affiche métriques sexe_conducteur
print_metrics(clf, X_test, y_test, df, categorical_features)

sexe_conducteur 0
disparate_impact 1.0028235328213417 0.6355715740529334
P_rule_disparate_impact 0.9971844170694738 0.6355715740529334
demography_parity 0.0008044019750252929 -0.022439373005842936


Les femmes conductrices impliquées dans un accident mortel : 0.6677316293929713 
Les femmes conductrices impliquées dans un accident non mortel : 0.2701366297983084 
Les hommes conducteurs impliquées dans un accident mortel : 0.6289855072463768 
Les hommes conducteurs impliquées dans un accident non mortel : 0.26231456827691135 

